In [2]:
import pandas as pd

### Preprocessing

In [3]:
# Used to convert survey flow id to study conditions. FL/ML: language, C: control, X, Y, Z: interventions, 1, 2: each condition has 2 orders
condition_mapping = {
    'FL_483': 'FL_C_1',
    'FL_484': 'FL_C_2',
    'FL_498': 'FL_X_1',
    'FL_510': 'FL_X_2',
    'FL_491': 'FL_Y_1',
    'FL_494': 'FL_Y_2',
    'FL_513': 'FL_Z_1',
    'FL_505': 'FL_Z_2',
    'FL_518': 'ML_C_1',
    'FL_523': 'ML_C_2',
    'FL_534': 'ML_X_1',
    'FL_539': 'ML_X_2',
    'FL_526': 'ML_Y_1',
    'FL_530': 'ML_Y_2',
    'FL_546': 'ML_Z_1',
    'FL_550': 'ML_Z_2',
}

# Used to convert competitiveness rating to numerical 
competitive_mapping = {
'Extremely uncompetitive': 1,
'Uncompetitive': 2,
'Somewhat uncompetitive': 3, 
'Neutral (neither competitive nor uncompetitive)':4,
'Somewhat competitive': 5,
'Competitive': 6,
'Extremely competitive': 7
}


In [4]:
def question_mapping(language, intervention, order):
    letter1_rating_index = language + '_' + order + '_' + 'C' + '_1' 
    letter1_confidence_index = language + '_' + order + '_' + 'C' + '_2' 
    order = '1' if order == '2' else '2'      
    letter2_rating_index = language + '_' + order + '_' + intervention + '_1' 
    letter2_confidence_index = language + '_' + order + '_' + intervention + '_2' 
    return letter1_rating_index, letter1_confidence_index, letter2_rating_index, letter2_confidence_index

def order2id(row):
    if row['Language'] == 'FL':
        return int(row['Order'])
    else:
        return int(row['Order']) + 2
    
def order2id2(row):
    if row['Order'] == '1':
        return row['LetterID'] + 1
    else:
        return row['LetterID'] -1

In [5]:
df = pd.read_csv('Responses/full_study.csv',header=2)  
df['Condition'] = df['Condition'].apply(lambda x: condition_mapping[x])
df['Language'] = df['Condition'].apply(lambda x: x.split('_')[0])
df['Intervention'] = df['Condition'].apply(lambda x: x.split('_')[1])
df['Order'] = df['Condition'].apply(lambda x: x.split('_')[2])

df_groups = []
df_groupby_condition = df.groupby('Condition')

# for each condition, get the ratings and confidence for the letters participants read
for condition in df['Condition'].unique():
    group =  df_groupby_condition.get_group(condition)
    group = group.dropna(axis='columns', how='all')
   
    letter1_rating_index, letter1_confidence_index, letter2_rating_index, letter2_confidence_index = question_mapping(condition.split('_')[0], condition.split('_')[1], condition.split('_')[2])

    group['letter1_rating'] = group[letter1_rating_index].apply(lambda x: competitive_mapping[x])
    group['letter1_confidence'] = group[letter1_confidence_index]
    group['letter2_rating'] = group[letter2_rating_index].apply(lambda x: competitive_mapping[x])
    group['letter2_confidence'] = group[letter2_confidence_index]


    df_groups.append(group[['ResponseId',  'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q96', 'IAT',
       'Condition', 'Language', 'Intervention', 'Order', 'letter1_rating',
       'letter1_confidence', 'letter2_rating', 'letter2_confidence']])

# concat all conditions
df_all_ratings = df_groups[0]
for i in range(1, len(df_groups)):
    df_all_ratings = pd.concat([df_all_ratings, df_groups[i]])


df_all_ratings['LetterID'] = df_all_ratings.apply(lambda x: order2id(x), axis=1)
df_all_ratings['LetterID2'] = df_all_ratings.apply(lambda x: order2id2(x), axis=1)

df1 = df_all_ratings[(df_all_ratings['LetterID']==1) & df_all_ratings['Order'] ==1]
mean1 = df1['letter1_rating'].mean()
df2 = df_all_ratings[(df_all_ratings['LetterID']==2) & df_all_ratings['Order'] ==1]
mean2 = df2['letter1_rating'].mean()
df3 = df_all_ratings[(df_all_ratings['LetterID']==3) & df_all_ratings['Order'] ==1]
mean3 = df3['letter1_rating'].mean()
df4 = df_all_ratings[(df_all_ratings['LetterID']==4) & df_all_ratings['Order'] ==1]
mean4 = df4['letter1_rating'].mean()

def get_mean(x):
    if x==1:
        return mean1
    if x==2:
        return mean2
    if x==3:
        return mean3
    if x==4:
        return mean4


df_all_ratings['letter2_control_mean'] = df_all_ratings['LetterID2'].apply(lambda x: get_mean(x))
df_all_ratings['letter1_control_mean'] = df_all_ratings['LetterID'].apply(lambda x: get_mean(x))

# save the results
df_all_ratings.to_csv('Responses/full_study_ratings.csv', index=False) 

In [6]:
# further process the data to have 1 rating each row
df_ratings1 = df_all_ratings[['ResponseId',  'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q96',
       'IAT', 'Condition', 'Language', 'Intervention', 'Order',
       'letter1_rating', 'letter1_confidence']]

df_ratings1 = df_ratings1.rename(columns={"letter1_rating": "rating", "letter1_confidence": "confidence"})

df_ratings1['LetterID'] = df_ratings1['Order']
df_ratings1['Intervention'] = 'C'

df_ratings2 = df_all_ratings[['ResponseId',  'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q96',
       'IAT', 'Condition', 'Language', 'Intervention', 'Order',
       'letter2_rating', 'letter2_confidence']]

df_ratings2['LetterID'] = df_ratings2['Order'].apply(lambda x: '1' if x=='2' else '2')

df_ratings2 = df_ratings2.rename(columns={"letter2_rating": "rating", "letter2_confidence": "confidence"})

df_ratings = pd.concat([df_ratings1, df_ratings2])

df_ratings.to_csv('Responses/full_study_ratings_per_letter.csv', index=False) 

/var/folders/3g/qxwb87wj1tx9vtln_mh1b2800000gr/T/ipykernel_42364/525388728.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings2['LetterID'] = df_ratings2['Order'].apply(lambda x: '1' if x=='2' else '2')


### Competitivenss rating analysis

#### Descriptive stastistics

In [7]:
# trial 1 rating
df_all_ratings.groupby(['Language', 'LetterID'])['letter1_rating'].describe()

count      mean       std  min  25%  50%  75%  max
Language LetterID                                                    
FL       1         137.0  5.503650  1.157610  1.0  5.0  6.0  6.0  7.0
         2         140.0  5.221429  1.132056  2.0  5.0  5.0  6.0  7.0
ML       3         141.0  5.751773  0.854869  3.0  5.0  6.0  6.0  7.0
         4         142.0  5.605634  1.024229  1.0  5.0  6.0  6.0  7.0

In [8]:
# trial 2 rating
df_all_ratings.groupby([ 'Language', 'LetterID', 'Intervention',])['letter2_rating'].describe()

count      mean       std  min  25%  50%  75%  \
Language LetterID Intervention                                                  
FL       1        C              34.0  5.235294  1.016793  2.0  5.0  5.0  6.0   
                  X              37.0  5.081081  1.255618  2.0  4.0  5.0  6.0   
                  Y              31.0  5.516129  0.995690  3.0  5.0  6.0  6.0   
                  Z              35.0  5.428571  1.065107  3.0  5.0  6.0  6.0   
         2        C              35.0  5.400000  1.116718  2.0  5.0  6.0  6.0   
                  X              34.0  5.205882  1.066839  3.0  5.0  5.0  6.0   
                  Y              35.0  5.600000  1.217519  3.0  5.0  6.0  6.0   
                  Z              36.0  5.361111  1.396992  1.0  4.0  6.0  6.0   
ML       3        C              35.0  5.885714  0.795998  4.0  5.0  6.0  6.0   
                  X              35.0  5.257143  1.038745  3.0  5.0  5.0  6.0   
                  Y              37.0  5.567568  0.834684  3.0  5.0  6.0  6.0   
                  Z              34.0  5.970588  0.797165  4.0  6.0  6.0  6.0   
         4        C              34.0  6.000000  0.921132  3.0  6.0  6.0  7.0   
                  X              34.0  5.382353  1.231277  2.0  5.0  6.0  6.0   
                  Y              37.0  5.648649  0.888701  3.0  5.0  6.0  6.0   
                  Z              37.0  5.837838  0.986394  2.0  5.0  6.0  6.0   

                                max  
Language LetterID Intervention       
FL       1        C             7.0  
                  X             7.0  
                  Y             7.0  
                  Z             7.0  
         2        C             7.0  
                  X             7.0  
                  Y             7.0  
                  Z             7.0  
ML       3        C             7.0  
                  X             7.0  
                  Y             7.0  
                  Z             7.0  
         4        C             7.0  
                  X             7.0  
                  Y             7.0  
                  Z             7.0

### Intervention Influence

In [9]:
# get the columns that is the influenced decision question 
influence_df = pd.DataFrame(columns=['ResponseId', 'Influenced', 'Intervention'])
for column in df.columns:
    if len(column.split('_')) > 0:
        if column.split('_')[-1] == '4':
            df_temp = df[['ResponseId', column, 'Intervention']].rename(columns={column: "Influenced"})
            if influence_df.shape[0] == 0:
                influence_df = df_temp.dropna()
            else:
                influence_df = pd.concat([influence_df, df_temp.dropna()]) 

influence_mapping = {
    'Much lower': -3,
    'Moderately lower': -2,
    'Slightly lower': -1,
    'About the same': 0,
    'Slightly higher': 1,
    'Moderately higher': 2,
    'Much higher': 3,
}

influence_df['Influenced_numerical'] = influence_df['Influenced'].apply(lambda x: influence_mapping[x]) 

influence_df_sub = influence_df[influence_df['Influenced_numerical']!=0]
influence_df_sub.groupby('Intervention').describe()

Influenced_numerical                                           \
                            count      mean       std  min  25%  50%   75%   
Intervention                                                                 
X                            62.0  0.596774  1.384453 -3.0 -1.0  1.0  1.75   
Y                            38.0  0.684211  1.416224 -3.0 -0.5  1.0  2.00   
Z                            61.0  1.065574  1.152806 -1.0  1.0  1.0  2.00   

                   
              max  
Intervention       
X             3.0  
Y             3.0  
Z             3.0